In [1]:
from Preprocess import *
from Model import *
from DataAnalysis import *

In [2]:
dic_csv = read_data()
df_var = make_dataframe(dic_csv)

In [3]:
concat_train_test(dic_csv, df_var)
loans = Merge(df_var, haveCategorical=False)

In [4]:
handle_non_numeric_data(loans)
original_loan_per_year = loan_per_year(loans, haveCategorical= False, isMerged = True)

In [5]:
df = original_loan_per_year[1993].copy()
X_train = df.drop(columns=['status'], axis=1)
y_train = df['status'].copy()
best_score, best_params = ClassifierDecisionTree(X_train, y_train)

: 15, &#39;classification__max_features&#39;: &#39;sqrt&#39;, &#39;classification__min_impurity_split&#39;: 0.05, &#39;classification__min_samples_leaf&#39;: 1, &#39;classification__min_samples_split&#39;: 2, &#39;classification__splitter&#39;: &#39;best&#39;}, {&#39;classification__class_weight&#39;: None, &#39;classification__criterion&#39;: &#39;entropy&#39;, &#39;classification__max_depth&#39;: 15, &#39;classification__max_features&#39;: &#39;sqrt&#39;, &#39;classification__min_impurity_split&#39;: 0.05, &#39;classification__min_samples_leaf&#39;: 1, &#39;classification__min_samples_split&#39;: 4, &#39;classification__splitter&#39;: &#39;best&#39;}, {&#39;classification__class_weight&#39;: None, &#39;classification__criterion&#39;: &#39;entropy&#39;, &#39;classification__max_depth&#39;: 15, &#39;classification__max_features&#39;: &#39;sqrt&#39;, &#39;classification__min_impurity_split&#39;: 0.05, &#39;classification__min_samples_leaf&#39;: 1, &#39;classification__min_samples_split&

In [10]:
report(cv_results)

,Mean absolute error,std,classification__class_weight,classification__criterion,classification__max_depth,classification__max_features,classification__min_impurity_split,classification__min_samples_leaf,classification__min_samples_split,classification__splitter
0,NaN,NaN,balanced,gini,2,auto,0.05,1,2,best
1,NaN,NaN,None,gini,11,auto,0.30,1,2,best
2,NaN,NaN,None,gini,11,auto,0.23,6,6,best


In [38]:
#df = loans[loans['status'] != 'none'].copy()
#X_train = df.drop(columns=['status', 'year', 'loan_id', 'account_id', "unemploymant rate '95 ", 'card_type', 'name', 'operation_n_credit_card_withdrawal'], axis=1)
#y_train = df['status'].copy()
best_score, best_params, cv_results = ClassifierDecisionTree(X_train, y_train)

ication__min_samples_leaf&#39;: 1, &#39;classification__min_samples_split&#39;: 4, &#39;classification__splitter&#39;: &#39;best&#39;}, {&#39;classification__class_weight&#39;: None, &#39;classification__criterion&#39;: &#39;entropy&#39;, &#39;classification__max_depth&#39;: 15, &#39;classification__max_features&#39;: &#39;sqrt&#39;, &#39;classification__min_impurity_split&#39;: 0.05, &#39;classification__min_samples_leaf&#39;: 1, &#39;classification__min_samples_split&#39;: 6, &#39;classification__splitter&#39;: &#39;best&#39;}, {&#39;classification__class_weight&#39;: None, &#39;classification__criterion&#39;: &#39;entropy&#39;, &#39;classification__max_depth&#39;: 15, &#39;classification__max_features&#39;: &#39;sqrt&#39;, &#39;classification__min_impurity_split&#39;: 0.05, &#39;classification__min_samples_leaf&#39;: 2, &#39;classification__min_samples_split&#39;: 2, &#39;classification__splitter&#39;: &#39;best&#39;}, {&#39;classification__class_weight&#39;: None, &#39;classificati

In [39]:
best_score

0.7719850874700013

In [40]:
report(cv_results)

,Mean absolute error,std,classification__class_weight,classification__criterion,classification__max_depth,classification__max_features,classification__min_impurity_split,classification__min_samples_leaf,classification__min_samples_split,classification__splitter
0,-0.771985,0.151213,balanced,gini,7,auto,0.05,1,6,best
1,-0.771985,0.151213,balanced,gini,7,sqrt,0.05,1,6,best
2,-0.771985,0.151213,None,gini,7,auto,0.05,1,6,best
3,-0.771985,0.151213,None,gini,7,sqrt,0.05,1,6,best


In [33]:
classifier = DecisionTreeClassifier(class_weight="balanced", criterion='gini', max_depth=7, max_features=auto, min_impurity_split=0.05, min_samples_leaf=1, min_samples_split=6, splitter="best")
classifier = apply_sampling(classfier)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
fpr, tpr, _ = metrics.roc_curve(y_test, y_pred)
auc = metrics.auc(fpr, tpr)

auc_scores.append(auc)
confusion_matrixes.append(metrics.confusion_matrix(y_test, y_pred).ravel())

plot_roc_auc(fpr, tpr, auc)

array([[-1.04251719, -0.61241419, -0.70940817, ..., -0.35301767,
        -0.31698156, -0.41283819],
       [ 1.10113431, -1.09464432, -1.42757447, ..., -0.04199366,
        -0.69016874,  1.39092295],
       [ 1.1088339 ,  0.19651742, -0.70940817, ...,  1.11323839,
         1.39967951,  0.61788246],
       ...,
       [ 0.10967967,  2.61280664,  1.44509072, ..., -1.28608972,
        -0.61553131, -1.18587868],
       [ 1.14089797, -0.86865961, -1.42757447, ...,  1.3798304 ,
         1.84750413,  1.13324279],
       [ 1.18530248, -0.15130947, -0.70940817, ...,  0.13573435,
         2.29532876, -0.02631794]])

In [43]:

#normalize_columns(X_train, X_train.columns)
df = loans[loans['status'] == 'none'].copy()
X_test = df.drop(columns=['status', 'year', 'loan_id', 'account_id', "unemploymant rate '95 ", 'card_type', 'name', 'operation_n_credit_card_withdrawal'], axis=1)
normalize_columns(X_test, X_test.columns)

In [44]:
def plot_roc_auc(fpr, tpr, roc_auc):
    '''Plot the ROC-AUC curve'''
    fig, ax = plt.subplots()
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'w--')
    plt.xlim([-0.02, 1.02])
    plt.ylim([0, 1])
    ax.fill_between(fpr, 0, tpr)
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

,loan_date,amount,duration,payments,birth_number,gender,client_district,account_district,frequency,account_date,region,no. of inhabitants,no. of municipalities with inhabitants < 499,no. of municipalities with inhabitants 500-1999,no. of municipalities with inhabitants 2000-9999,no. of municipalities with inhabitants >10000,no. of cities,ratio of urban inhabitants,average salary,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96,trans_date_min,trans_date_max,trans_amount_min,trans_amount_max,trans_amount_mean,trans_amount_std,trans_amount_last,trans_balance_min,trans_balance_max,trans_balance_mean,trans_balance_std,trans_balance_last,trans_type_n_credit,trans_type_n_withdrawal,operation_n_credit_cash,operation_n_collection_from_another_bank,operation_n_withdrawal_in_cash,operation_n_remittance_to_another_bank,operation_n_interest_credited
2,0.099505,0.042905,0.00,0.231001,0.598158,1.0,0.881579,0.881579,1.0,0.495554,0.428571,0.160074,0.099338,0.571429,0.90,0.4,0.5,0.352496,0.402392,0.478261,0.174419,0.065641,0.050896,0.495554,0.099514,0.024008,0.778639,0.415043,0.600006,0.017255,0.476817,0.671892,0.777355,0.581586,0.238091,0.583333,0.676647,0.795455,0.000000,0.729167,0.511905,0.312500
3,0.982530,0.534996,1.00,0.520924,0.377153,1.0,0.250000,0.250000,1.0,0.980580,0.285714,0.023943,0.622517,0.200000,0.15,0.2,0.3,0.370651,0.098623,0.357860,0.453488,0.018254,0.006659,0.980580,0.982619,0.000333,0.569521,0.236311,0.434065,0.174675,0.425993,0.580564,0.466672,0.613074,0.128676,0.357143,0.502994,0.386364,0.000000,0.500000,0.428571,0.291667
4,0.928951,0.180285,0.75,0.208516,0.108753,0.0,0.236842,0.236842,0.5,0.980099,0.285714,0.007406,0.331126,0.157143,0.15,0.2,0.3,0.284418,0.065899,0.395764,0.453488,0.011672,0.002148,0.980099,0.928404,0.049683,0.195656,0.123931,0.167410,0.003542,0.820952,0.281452,0.351089,0.209474,0.279657,0.250000,0.191617,0.113636,0.391304,0.197917,0.154762,0.166667
6,0.063665,0.166865,0.00,0.860816,0.153224,1.0,0.960526,0.960526,1.0,0.484834,0.428571,0.241839,0.000000,0.000000,0.00,0.2,0.0,1.000000,0.578425,0.558528,0.220930,0.219228,0.177756,0.484834,0.063581,0.004668,0.280465,0.128962,0.219867,0.000172,0.476817,0.308092,0.373664,0.265010,0.301890,0.369048,0.508982,0.386364,0.000000,0.437500,0.511905,0.312500
7,0.099325,0.444228,0.50,0.735582,0.710930,1.0,0.565789,0.565789,1.0,0.475460,1.000000,0.054026,0.509934,0.371429,0.35,0.4,0.6,0.288956,0.032498,0.371237,0.186047,0.025292,0.014631,0.475460,0.092669,0.007002,0.648589,0.278109,0.451502,0.000211,0.462962,0.649726,0.551804,0.786625,0.154678,0.476190,0.245509,0.477273,0.000000,0.427083,0.000000,0.416667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,0.099775,0.257161,0.25,0.643973,0.287991,0.0,0.421053,0.421053,1.0,0.946642,0.000000,0.078266,0.158940,0.242857,0.35,0.6,1.0,0.768533,0.134281,0.800446,0.406977,0.054285,0.040430,0.946642,0.099694,0.024008,0.566962,0.470924,0.444649,0.472666,0.458202,0.603728,0.628640,0.653614,0.408986,0.416667,0.275449,0.590909,0.000000,0.406250,0.083333,0.208333
675,0.002521,0.106155,0.50,0.163023,0.820351,1.0,0.052632,0.052632,1.0,0.024083,0.857143,0.045429,0.430464,0.428571,0.20,0.2,0.5,0.264750,0.270142,0.445931,0.430233,0.030545,0.021910,0.024083,0.000991,0.024008,0.141298,0.085889,0.105660,0.018660,0.458202,0.207600,0.285120,0.088947,0.152990,0.428571,0.437126,0.136364,0.695652,0.385417,0.428571,0.312500
676,0.992256,0.533254,1.00,0.519155,0.889274,1.0,0.644737,0.644737,1.0,0.975436,1.000000,0.051447,0.496689,0.471429,0.10,0.6,0.5,0.273828,0.017378,0.559643,0.151163,0.024790,0.012686,0.975436,0.992435,0.024008,0.762037,0.503213,0.650937,0.604281,0.471498,0.771479,0.804149,0.719527,0.560057,0.571429,0.568862,0.750000,0.000000,0.697917,0.333333,0.333333
677,0.974696,0.058087,0.25,0.138247,0.976228,0.0,0.078947,0.078947,1.0,0.990050,0.857143